# Air netCDF File for All Variables (mean, clim, anom,std)

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta

# air

In [2]:
os.chdir('d:/NCEP_R2')
ds_air= xr.open_dataset('air.2m.mon.mean.nc') # ds_precip

In [5]:
## unit change
ds_air['air'] = ds_air.air-273.15

## change dim
ds_air = ds_air.squeeze('level')
ds_air = ds_air.drop('level', dim=None)

ds_air # check

<xarray.Dataset>
Dimensions:  (time: 545, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2024-05-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    air      (time, lat, lon) float32 -33.48 -33.51 -33.53 ... -45.3 -45.48

### cut new clim_prate
- air
- clim_ds
- anom_ds
- std_ds

In [11]:
new_clim = ds_air.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    air      (time, lat, lon) float32 -24.83 -24.6 -24.53 ... -24.54 -24.63

In [12]:
lon = new_clim.lon
lat = new_clim.lat
time = new_clim.time

#### ◆ air nc file (monthly mean)

In [14]:
os.getcwd()
new_clim.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
new_clim.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
new_clim.air.attrs ={'unit' : 'deg C', 'long_name':'NCEP Monthly Air Temperature at 2m'}

# new_clim.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_mon_mean_new.nc')

#### ◆ air clim nc file

In [15]:
clim_sel = new_clim['air'].groupby('time.month').mean('time')


clim_extend_lst = []
for i in range(30):
    for j in clim_sel.values:
        clim_extend_lst.append(j)
        
clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars = dict(
        new_clim = (['time','lat','lon'], clim_extend_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.new_clim.attrs ={'units':'deg C','long_name':'Average Long Term Mean Monthly Rate of Air Temperature at 2m '}

In [16]:
# clim_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_clim_new.nc')

#### ◆ air anom nc file

In [17]:
anom_lst = []
for i in range(1,13):
    each_mon = new_clim['air'].sel(time=new_clim['time.month']==i) # month
    # clim_mon = clim_sel.sel(month=i) # clim # new_clim['air'].groupby('time.month').mean('time')
    clim_mon = clim_ds['new_clim'].sel(time=clim_ds['time.month']==i) 
    
    anom_sel = each_mon - clim_mon # anom
    anom_lst.append(anom_sel) 
    
anom_cb  = xr.concat(anom_lst, dim='time')

anom_ds = xr.Dataset(
    data_vars = dict(
        new_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
anom_ds.lon.attrs = new_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.new_anom.attrs ={'units':'deg C','long_name':'The Monthly Anomaly Rate of Forecasted Air Temperature at 2m'} 
anom_ds


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


<xarray.Dataset>
Dimensions:   (time: 360, lon: 192, lat: 94)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    new_anom  (time, lat, lon) float32 1.916 1.996 2.099 ... 2.349 2.357 2.367

In [18]:
# anom_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_anom_new.nc')

#### ◆ air std nc file

In [19]:
## std
std_sel=new_clim.air.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        new_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.new_std.attrs ={'units':'deg C','long_name':'Standard Deviation Rate of Forecasted Air Temperature at 2m'}

In [20]:
os.getcwd()
# std_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_std_new.nc')

### cut old clim_prate
- air
- clim_ds
- anom_ds
- std_ds

In [21]:
old_clim = ds_air.sel(time=slice('1981-01-01','2010-12-01'))
old_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    air      (time, lat, lon) float32 -30.21 -30.08 -30.82 ... -27.47 -27.54

In [22]:
lon = old_clim.lon
lat = old_clim.lat
time = old_clim.time

#### ◆ air nc file (monthly mean)

In [23]:
os.getcwd()
old_clim.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
old_clim.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
old_clim.air.attrs ={'unit' : 'deg C', 'long_name':'NCEP Monthly Air Temperature at 2m'}

# old_clim.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_mon_mean_old.nc')

#### ◆ air clim nc file

In [24]:
clim_sel = old_clim['air'].groupby('time.month').mean('time')


clim_extend_lst = []
for i in range(30):
    for j in clim_sel.values:
        clim_extend_lst.append(j)
        
clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars = dict(
        old_clim = (['time','lat','lon'], clim_extend_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.old_clim.attrs ={'units':'deg C','long_name':'Average Long Term Mean Monthly Rate of Air Temperature at 2m '}

In [25]:
# clim_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_clim_old.nc')

#### ◆ air anom nc file

In [26]:
anom_lst = []
for i in range(1,13):
    each_mon = old_clim['air'].sel(time=old_clim['time.month']==i) # month
    # clim_mon = clim_sel.sel(month=i) # clim # old_clim['air'].groupby('time.month').mean('time')
    clim_mon = clim_ds['old_clim'].sel(time=clim_ds['time.month']==i) 
    
    anom_sel = each_mon - clim_mon # anom
    anom_lst.append(anom_sel) 
    
anom_cb  = xr.concat(anom_lst, dim='time')

anom_ds = xr.Dataset(
    data_vars = dict(
        old_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
anom_ds.lon.attrs = old_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.old_anom.attrs ={'units':'deg C','long_name':'The Monthly Anomaly Rate of Forecasted Air Temperature at 2m'} 
anom_ds


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


<xarray.Dataset>
Dimensions:   (time: 360, lon: 192, lat: 94)
Coordinates:
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    old_anom  (time, lat, lon) float32 -2.309 -2.262 -2.979 ... -0.474 -0.4473

In [27]:
# anom_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_anom_old.nc')

#### ◆ air std nc file

In [28]:
## std
std_sel=old_clim.air.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        old_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.old_std.attrs ={'units':'deg C','long_name':'Standard Deviation Rate of Forecasted Air Temperature at 2m'}

In [29]:
os.getcwd()
# std_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/air/NCEP_air_2m_std_old.nc')